In [3]:
import numpy as np
import pandas as pd
import json
import uuid
import os
from time import time_ns

In [4]:
str(uuid.uuid1())

'359d0c60-d98a-11ed-baa8-e8d63159f8ff'

In [10]:
# add a question mark before or after a function to print the docstring
np.random.randn?
# uuid.uuid1?

In [5]:
def make_test_meta():
    test_meta = {'uuid': str(uuid.uuid1()),
                'param1':12,
                'param2':'a_string',
                'param3': np.random.rand()*1e9,
                'param4':['alist','of','strings']}
    return test_meta


# time is in nano seconds? generealy, about 1200 datapoints
def make_test_data(length_of_data):
    test_data = {'time':(np.array(range(length_of_data))*1e-9).tolist(), # to_list is necessary because json.dumps only takes native python types not numpy types
                'vals': np.random.randn(length_of_data).tolist()}
    return test_data

def make_json_datastore(testdata, meta=None):
    if meta is None:
        test_meta = make_test_meta()
    metadata_as_string = json.dumps(test_meta)
    kepler_json_format = {metadata_as_string: testdata}
    return kepler_json_format

def write_n_to_file(length_of_data, n_to_write, fp = None):
    if fp is None:
        fp = 'testfile.json'
    if os.path.exists(fp):
        os.remove(fp)
    then = time_ns()
    with open(fp,'w+') as f:
        for i in range(n_to_write):
            jsonified_data = make_json_datastore(make_test_data(length_of_data)) 
            to_write = json.dumps(jsonified_data)
            f.write(to_write+'\r')
    now = time_ns()
    return (now-then) * 1e-9

def jsonlike_load(source):
    then = time_ns()
    with open(source, 'r') as f:
        lines = f.readlines()

    metajson = {}
    for line in lines:
        subjson = json.loads(line.rstrip(r'\n'))
        for key, item in subjson.items():
            metajson[key] = item
    now = time_ns()
    return metajson, (now-then)*1e-9

In [6]:
fp = 'testfile.json'
# does 100 represent the number of experiments and 1200 is the no of nanoseconds, so each 1 experiment has 1200 datapoints
writetime = write_n_to_file(1200,100, fp)
out, readtime = jsonlike_load('testfile.json')
print(f'time to write: {writetime} s, time to read {readtime} s ')

time to write: 0.5070342 s, time to read 0.24158900000000003 s 
